In [1]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 20000
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)

In [3]:
max_len = 750
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [4]:
print(X_train.shape)
print(X_test.shape)

(25000, 750)
(25000, 750)


In [5]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [6]:
embedding_dim = 256
dropout_ratio = 0.3
num_filters = 256 #count of kernel
kernel_size = 3
hidden_units = 128

In [7]:
inputs = Input(shape=(max_len))
model = Embedding(vocab_size, embedding_dim)(inputs)
model = Dropout(dropout_ratio)(model)
model = Conv1D(num_filters, kernel_size, padding="valid", activation="relu")(model)
model = GlobalMaxPooling1D()(model)
model = Dense(hidden_units, activation="relu")(model)
model = Dropout(dropout_ratio)(model)
output = Dense(1, activation="sigmoid")(model)
model = Model(inputs=inputs, outputs=output)

'inputs = Input(shape=(max_len))\nmodel = Embedding(vocab_size, embedding_dim)(inputs)\nmodel = Dropout(dropout_ratio)(model)\nmodel = Conv1D(num_filters, kernel_size, padding="valid", activation="relu")(model)\nmodel = GlobalMaxPooling1D()(model)\nmodel = Dense(hidden_units, activation="relu")(model)\nmodel = Dropout(dropout_ratio)(model)\noutput = Dense(1, activation="sigmoid")(model)\nmodel = Model(inputs=inputs, outputs=output)'

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('Models/IMDB_1DCNN_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 dropout (Dropout)           (None, None, 256)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 256)         196864    
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
